## Configuration

In [ ]:
TRAIN_PATH = 'data/ebay/Tagged_Titles_Train.tsv'
LIST_PATH = 'data/ebay/Listing_Titles.tsv'

SAVE_DIRECTORY = "./saved_models"
EPOCHS = 30
BATCH_SIZE = 64
FINE_TUNE_BERT = False # set to true to train BERT as well
LR = 2e-5 if FINE_TUNE_BERT else 5e-4
MODEL_NAME = "GottBERT/GottBERT_base_last"


## Initialization

In [2]:
import csv 
import pandas as pd
import numpy as np
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoModel, AutoConfig
import matplotlib.pyplot as plt
import seaborn as sns
import traceback 
import numpy as np
import os
from tqdm import tqdm
# Load model directly
from transformers import AutoTokenizer, AutoModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE=device
print(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(DEVICE)
tokenizer = tokenizer

tagged_train = pd.read_csv(TRAIN_PATH, sep='\t')
os.makedirs(SAVE_DIRECTORY, exist_ok=True)




/home/manjot/Files/Code/Ebay/venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


Loading weights: 100%|██████████| 199/199 [00:00<00:00, 284.05it/s, Materializing param=pooler.dense.weight]                               
RobertaModel LOAD REPORT from: GottBERT/GottBERT_base_last
Key                       | Status     |  | 
--------------------------+------------+--+-
lm_head.dense.bias        | UNEXPECTED |  | 
lm_head.dense.weight      | UNEXPECTED |  | 
lm_head.bias              | UNEXPECTED |  | 
lm_head.layer_norm.bias   | UNEXPECTED |  | 
lm_head.decoder.bias      | UNEXPECTED |  | 
lm_head.layer_norm.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


## MLP Classifier Architecture 

In [3]:
import torch
import torch.nn as nn

class MLPClassifier(nn.Module):
    def __init__(self, input_dim=768, hidden_dims=[64, 256, 128], num_classes=59, dropout=0.3):
        """
        Args:
            input_dim (int): Size of input features.
            hidden_dims (list): List of hidden layer sizes. Pass [] for a single linear layer.
            num_classes (int): Number of output classes.
            dropout (float): Dropout probability.
        """
        super().__init__()
        layers = []
        current_dim = input_dim

        # Dynamically create hidden layers
        for h_dim in hidden_dims:
            layers.append(nn.Linear(current_dim, h_dim))
            layers.append(nn.LayerNorm(h_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            current_dim = h_dim

        # Final output layer (Project to num_classes)
        layers.append(nn.Linear(current_dim, num_classes))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        logits = self.model(x)
        return logits

In [4]:
def get_original_classes(df):
    tags = df['Tag'].fillna("").astype(str).str.strip()
    unique_tags = tags[tags != ''].unique()
    sorted_tags = sorted(unique_tags)
    print(f"--- Original Class Analysis ---")
    print(f"Total Unique Classes Found: {len(sorted_tags)}")
    print(f"Classes: {sorted_tags}")
    
    return sorted_tags

print("done")

done


## Preprocessing

In [5]:
from torch.utils.data import Dataset

def preprocess_ebay_ner_data(df):
    df['Tag'] = df['Tag'].fillna('')
    all_raw_tags = df['Tag'].astype(str).str.strip().unique()
    unique_aspects = set()
    
    for tag in all_raw_tags:
        if tag not in ['', 'O', '0', 'nan']:
            unique_aspects.add(tag)
            
    # 3. Construct the full label set (O + B-Tag + I-Tag for every aspect)
    # This ensures 59 classes (1 'O' + 29*2) if all 29 aspects are in the data.
    unique_labels = ['O']
    for aspect in sorted(list(unique_aspects)):
        unique_labels.append(f"B-{aspect}")
        unique_labels.append(f"I-{aspect}")
        
    # Create mappings
    # We sort again to ensure 'O' and the rest are in a deterministic order
    unique_labels = sorted(unique_labels)
    label2id = {label: i for i, label in enumerate(unique_labels)}
    id2label = {i: label for i, label in enumerate(unique_labels)}

    print(f"Generated {len(unique_labels)} classes from data.")
    grouped_titles = []
    grouped_labels = []
    grouped_categories = []


    for record_id, group in df.groupby('Record Number'):
        tokens = group['Token'].astype(str).tolist()
        raw_tags = group['Tag'].tolist()
        raw_category = group['Category'].iloc[0]
        category_val = int(raw_category) -1
        bio_tags = []
        current_entity_tag = None
        for tag in raw_tags: 
            tag = tag.strip()
            if tag == 'O' or tag == '0': 
                bio_tags.append('O')
                current_entity_tag = None
            elif tag != '':
                bio_tags.append(f"B-{tag}")
                current_entity_tag = tag
            else:
                if current_entity_tag:
                    bio_tags.append(f"I-{current_entity_tag}")
                else:
                    bio_tags.append('O')

        grouped_titles.append(tokens)
        grouped_labels.append(bio_tags)
        grouped_categories.append(category_val)
    # unique_labels = sorted(list(set([l for seq in grouped_labels for l in seq])))
    label2id = {label: i for i, label in enumerate(unique_labels)}
    id2label = {i: label for i, label in enumerate(unique_labels)}

    return grouped_titles, grouped_labels,grouped_categories, label2id, id2label 


class BertDataset(Dataset):
    def __init__(self, token_lists, label_lists, category_lists, label2id, max_len=128):
        self.token_lists = token_lists
        self.label_lists = label_lists
        self.category_lists = category_lists
        self.label2id = label2id
        self.tokenizer = tokenizer
        self.max_len = max_len 

    def __len__(self):
        return len(self.token_lists)

    def __getitem__(self, idx):
        word_list = self.token_lists[idx]
        label_list = self.label_lists[idx]
        category_id = self.category_lists[idx]
        encoding = self.tokenizer(
            word_list,
            is_split_into_words=True,
            padding='max_length',
            truncation =True, 
            max_length=self.max_len,
            return_tensors='pt'
        )
        word_ids = encoding.word_ids(batch_index=0)
        encoded_labels= []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                encoded_labels.append(-100)
            elif word_idx != previous_word_idx:
                label_str = label_list[word_idx]
                encoded_labels.append(self.label2id[label_str])

            else: 
                encoded_labels.append(-100)
            
            previous_word_idx = word_idx 

        labels_tensor = torch.tensor(encoded_labels, dtype=torch.long)
        item = {key: val.squeeze() for key, val in encoding.items()}
        item['labels'] = labels_tensor
        item['category'] = torch.tensor(category_id, dtype=torch.float32)

        return item
        





In [6]:
import pandas as pd
import csv
import os
from sklearn.model_selection import train_test_split

def load_and_prepare_data(train_csv_path, listing_csv_path=None):
    """
    Loads the eBay NER datasets with specific parameters to handle 
    tab-separation and empty 'continuation' tags correctly.
    
    Args:
        train_csv_path (str): Path to the tagged training data file.
        listing_csv_path (str, optional): Path to the listing titles file (if needed for context).
        
    Returns:
        tuple: (train_df, token_lists, label_lists, label2id, id2label)
    """
    print(f"Loading data from {train_csv_path}...")
    
    # 1. Load Training Data
    # source: [113-116] - Specific pandas settings required:
    # - sep="\t": Data is tab-separated
    # - keep_default_na=False: Prevent pandas from turning empty strings into NaN
    # - na_values=None: Ensure no other values are treated as NA
    # - quoting: The file uses CSV-style quoting 
    try:
        train_df = pd.read_csv(
            train_csv_path, 
            sep="\t", 
            keep_default_na=False, 
            na_values=None,
            quoting=csv.QUOTE_MINIMAL, # Handles standard CSV quoting used in the file
            encoding='utf-8'
        )
    except Exception as e:
        print(f"Error loading train data: {e}")
        return None

    # 2. Load Listing Data (Optional, context only)
    # The training file already contains tokens, but this can be useful for EDA.
    if listing_csv_path and os.path.exists(listing_csv_path):
        listing_df = pd.read_csv(
            listing_csv_path, 
            sep="\t", 
            keep_default_na=False, 
            na_values=None,
            encoding='utf-8'
        )
        print(f"Loaded {len(listing_df)} listing records.")
    
    print(f"Loaded {len(train_df)} training tokens.")

    # 3. Preprocess to BIO Format
    # Re-using the logic defined in previous steps
    print("Preprocessing data into BIO format...")
    labels = get_original_classes(train_df)
    token_lists, label_lists, category_lists,label2id, id2label = preprocess_ebay_ner_data(train_df)
    
    print(f"Processed {len(token_lists)} sentences.")
    print(f"Found {len(label2id)} unique classes (including 'O' and B-/I- prefixes).")
    
    return train_df, token_lists, label_lists, category_lists, label2id, id2label

In [7]:
train_df, grouped_titles, grouped_labels, grouped_categories, label2id, id2label = load_and_prepare_data(TRAIN_PATH, LIST_PATH)

Loading data from data/ebay/Tagged_Titles_Train.tsv...
Loaded 56812 training tokens.
Preprocessing data into BIO format...
--- Original Class Analysis ---
Total Unique Classes Found: 30
Classes: ['Anwendung', 'Anzahl_Der_Einheiten', 'Besonderheiten', 'Breite', 'Bremsscheiben-Aussendurchmesser', 'Bremsscheibenart', 'Einbauposition', 'Farbe', 'Größe', 'Hersteller', 'Herstellernummer', 'Herstellungsland_Und_-Region', 'Im_Lieferumfang_Enthalten', 'Kompatible_Fahrzeug_Marke', 'Kompatibles_Fahrzeug_Jahr', 'Kompatibles_Fahrzeug_Modell', 'Länge', 'Material', 'Maßeinheit', 'Menge', 'Modell', 'O', 'Oberflächenbeschaffenheit', 'Oe/Oem_Referenznummer(N)', 'Produktart', 'Produktlinie', 'SAE_Viskosität', 'Stärke', 'Technologie', 'Zähnezahl']
Generated 59 classes from data.
Processed 5000 sentences.
Found 59 unique classes (including 'O' and B-/I- prefixes).


## Dataset Verification

In [8]:
total_dataset = BertDataset(token_lists = grouped_titles, label_lists = grouped_labels, category_lists=grouped_categories, label2id = label2id, max_len=128)
sample = total_dataset[1826]
print(sample)
print(tokenizer.convert_ids_to_tokens(sample['input_ids']))
print(sample['input_ids'].shape)

{'input_ids': tensor([    0,   305,   524,   796,   124,   319, 51454, 34499, 19833,  2924,
        37141,   546, 19343, 43388,  5684,  2118,   166,   199,  2656,   166,
        39988,   951,   219,   951,   380,   728,   380,     2,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1, 

In [9]:
example = "MINI 1.6 W10B16A W11B16A R50 Steuerkettensatz 11311485400 Steuerkette FEBI 31803"

In [14]:
tokens = tokenizer(example.split(" "), return_tensors="pt", padding=True, is_split_into_words=True)
tokens = tokens.to(DEVICE)
print(tokens.word_ids())
print(tokenizer.convert_ids_to_tokens(tokens["input_ids"][0]))
with torch.no_grad():
    outs = model(**tokens)

print(outs.last_hidden_state.shape)
classifier = MLPClassifier(input_dim=769, hidden_dims=[64], num_classes=59).to(DEVICE)
# preds = classifier(outs.last_hidden_state)
# print(preds.shape)

[None, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 8, 8, 9, 9, 9, None]
['<s>', 'M', 'IN', 'I', '1', '.', '6', 'W', '10', 'B', '16', 'A', 'W', '11', 'B', '16', 'A', 'R', '50', 'Steuer', 'ketten', 'satz', '113', '11', '48', '54', '00', 'Steuer', 'kette', 'FE', 'BI', '3', '18', '03', '</s>']
torch.Size([1, 35, 768])


## Training Functions

In [17]:
import traceback
def validate(bert_model, classifier, dataloader, device, id2label):
    bert_model.eval()
    classifier.eval()

    loss_fn = nn.CrossEntropyLoss(ignore_index=-100)
    running_loss = 0.0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_masks = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            categories = batch['category'].to(device)


            outputs = bert_model(input_ids, attention_mask=attention_masks)
            sequence_output = outputs.last_hidden_state
            # Prepare Category feature
            seq_len = sequence_output.shape[1]
            cat_expanded = categories.unsqueeze(1).unsqueeze(2).expand(-1, seq_len, 1)
            combined_features = torch.cat((sequence_output, cat_expanded), dim=2)

            logits = classifier(combined_features)

            loss = loss_fn(logits.view(-1, logits.shape[-1]), labels.view(-1))
            running_loss += loss.item()
            preds = torch.argmax(logits, dim=2)
            mask = labels != -100 
            all_preds.extend(preds[mask].cpu().numpy())
            all_labels.extend(labels[mask].cpu().numpy())
    avg_loss = running_loss / len(dataloader)
    unique_labels = sorted(list(set(all_labels)))
    target_names = [id2label[i] for i in unique_labels]
    report_dict = classification_report(
        all_labels,
        all_preds, 
        labels=unique_labels,
        target_names=target_names,
        output_dict=True,
        zero_division=0
    )
    report_str = classification_report(
        all_labels,
        all_preds, 
        labels=unique_labels,
        target_names=target_names,
        zero_division=0
    )
    return avg_loss, report_dict, report_str

def plot_results(train_loss_history, val_loss_history, final_report, epochs):
    """
    Plots training vs validation loss and per-class F1 scores.
    """
    plt.figure(figsize=(16, 6))

    # Plot 1: Loss Curves
    plt.subplot(1, 2, 1)
    plt.plot(train_loss_history, label='Train Loss', color='#1f77b4', linewidth=2)
    
    # We plot val loss as points or a line corresponding to epochs
    # Since val_loss is recorded once per epoch, we stretch it or plot it against epoch indices
    # Here we just plot the sequence of validation losses
    plt.plot(np.linspace(0, len(train_loss_history), len(val_loss_history)), 
             val_loss_history, label='Val Loss', color='#d62728', marker='o', linewidth=2)
    
    plt.title(f"Loss Curve over {epochs} Epochs")
    plt.xlabel("Training Steps")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Plot 2: Top 20 Class F1 Scores
    plt.subplot(1, 2, 2)
    class_metrics = {k: v['f1-score'] for k, v in final_report.items() 
                     if isinstance(v, dict) and k not in ['accuracy', 'macro avg', 'weighted avg']}
    
    # Sort and slice top 20
    sorted_classes = sorted(class_metrics.items(), key=lambda x: x[1], reverse=True)
    names = [x[0] for x in sorted_classes]
    scores = [x[1] for x in sorted_classes]
    
    sns.barplot(x=scores, y=names, palette='viridis', hue=names, legend=False)
    plt.title(f"Top F1 Scores (Epoch {epochs})")
    plt.xlabel("F1 Score")
    plt.xlim(0, 1.0)
    plt.grid(axis='x', alpha=0.3)

    plt.tight_layout()
    plt.show()

def train_model(bert_model, classifier, train_loader, val_loader, label2id, id2label):
    print(f"Initializing Models... (Fine-tune Bert: {FINE_TUNE_BERT})")
    for param in bert_model.parameters():
        param.requires_grad = FINE_TUNE_BERT
    
    params_to_optimize = list(classifier.parameters())
    if FINE_TUNE_BERT:
        params_to_optimize += list(bert_model.parameters())
    optimizer = torch.optim.AdamW(params_to_optimize, lr=LR)
    loss_fn = nn.CrossEntropyLoss(ignore_index = -100)

    train_loss_history = []
    val_loss_history = []
    final_val_report = {}
    try:
        print(f"Starting Training for {EPOCHS} Epochs... ")
        for epoch in range(EPOCHS):
            bert_model.train()
            classifier.train()
            running_train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"EPOCH {epoch + 1}/ {EPOCHS}")
            for batch in progress_bar: 
                input_ids = batch['input_ids'].to(DEVICE)
                attention_mask = batch['attention_mask'].to(DEVICE)
                labels = batch['labels'].to(DEVICE)
                categories = batch['category'].to(DEVICE)
                
                optimizer.zero_grad()
                
                # Forward Pass
                outputs = bert_model(input_ids, attention_mask=attention_mask)
                sequence_output = outputs.last_hidden_state
                seq_len = sequence_output.shape[1]
                cat_expanded = categories.unsqueeze(1).unsqueeze(2).expand(-1, seq_len, 1)
                combined_features = torch.cat((sequence_output, cat_expanded), dim=2)

                logits = classifier(combined_features)
                
                # Loss
                loss = loss_fn(logits.view(-1, len(label2id)), labels.view(-1))
                
                # Backward
                loss.backward()
                optimizer.step()
                
                # Track Loss
                running_train_loss += loss.item()
                train_loss_history.append(loss.item())
                
                # Update Progress Bar
                progress_bar.set_postfix({'loss': loss.item()})
            print(f"Validating Epoch {epoch+1}... ")
            val_loss, val_report, val_report_str = validate(bert_model, classifier, val_loader, DEVICE, id2label) # change this back to val
            val_loss_history.append(val_loss)
            final_val_report = val_report

            print(f"Epoch {epoch+1} Summary | Train Loss: {running_train_loss/len(train_loader):.4f} | Val Loss: {val_loss:.4f}")
            print(val_report_str)
    except Exception as e:
        raise

    finally:
        torch.save(bert_model.state_dict(), os.path.join(SAVE_DIRECTORY, "bert_model.pth"))
        torch.save(classifier.state_dict(), os.path.join(SAVE_DIRECTORY, "classifier_head.pth"))
        plot_results(train_loss_history, val_loss_history, final_val_report, EPOCHS)
    
    return bert_model, classifier, train_loss_history, val_loss_history, final_val_report



## Training Loop

In [ ]:

train_toks, val_toks, train_labs, val_labs, train_cats, val_cats = train_test_split(
    grouped_titles, grouped_labels, grouped_categories, test_size=0.2, random_state=42
)
train_ds = BertDataset(train_toks, train_labs, train_cats, label2id)
val_ds = BertDataset(val_toks, val_labs, val_cats, label2id)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)
bert_model, classifier, train_loss, val_loss, report = train_model(
    model, classifier, train_loader, val_loader, label2id, id2label
)



Initializing Models... (Fine-tune Bert: False)
Starting Training for 100 Epochs... 


EPOCH 1/ 100:   0%|          | 0/63 [00:00<?, ?it/s]

EPOCH 1/ 100: 100%|██████████| 63/63 [00:26<00:00,  2.38it/s, loss=1.33]


Validating Epoch 1... 
Epoch 1 Summary | Train Loss: 1.5309 | Val Loss: 1.0551
                                   precision    recall  f1-score   support

                      B-Anwendung       0.00      0.00      0.00         1
           B-Anzahl_Der_Einheiten       1.00      0.05      0.10        56
                 B-Besonderheiten       0.00      0.00      0.00        12
B-Bremsscheiben-Aussendurchmesser       0.80      0.91      0.85       253
               B-Bremsscheibenart       0.00      0.00      0.00        69
                 B-Einbauposition       0.92      0.98      0.95       564
                          B-Farbe       0.00      0.00      0.00         2
                          B-Größe       0.00      0.00      0.00         4
                     B-Hersteller       0.88      0.81      0.84       770
               B-Herstellernummer       0.58      0.30      0.39       264
      B-Im_Lieferumfang_Enthalten       0.79      0.92      0.85      1279
      B-Kompatible_F

EPOCH 2/ 100: 100%|██████████| 63/63 [00:26<00:00,  2.37it/s, loss=1.01] 


...

EPOCH 29/ 100: 100%|██████████| 63/63 [00:27<00:00,  2.32it/s, loss=0.433]


Validating Epoch 29... 
Epoch 29 Summary | Train Loss: 0.5192 | Val Loss: 0.4086
                                   precision    recall  f1-score   support

                      B-Anwendung       0.25      1.00      0.40         1
           B-Anzahl_Der_Einheiten       0.94      0.91      0.93        56
                 B-Besonderheiten       1.00      0.08      0.15        12
B-Bremsscheiben-Aussendurchmesser       0.87      0.98      0.92       253
               B-Bremsscheibenart       0.88      0.81      0.84        69
                 B-Einbauposition       0.93      0.99      0.96       564
                          B-Farbe       0.00      0.00      0.00         2
                          B-Größe       0.40      0.50      0.44         4
                     B-Hersteller       0.96      0.96      0.96       770
               B-Herstellernummer       0.87      0.84      0.85       264
      B-Im_Lieferumfang_Enthalten       0.91      0.93      0.92      1279
      B-Kompatible

EPOCH 30/ 100: 100%|██████████| 63/63 [00:27<00:00,  2.32it/s, loss=0.589]


Validating Epoch 30... 
Epoch 30 Summary | Train Loss: 0.5145 | Val Loss: 0.4067
                                   precision    recall  f1-score   support

                      B-Anwendung       0.33      1.00      0.50         1
           B-Anzahl_Der_Einheiten       0.93      0.91      0.92        56
                 B-Besonderheiten       1.00      0.08      0.15        12
B-Bremsscheiben-Aussendurchmesser       0.89      0.94      0.91       253
               B-Bremsscheibenart       0.87      0.80      0.83        69
                 B-Einbauposition       0.93      0.99      0.96       564
                          B-Farbe       0.00      0.00      0.00         2
                          B-Größe       0.40      0.50      0.44         4
                     B-Hersteller       0.96      0.96      0.96       770
               B-Herstellernummer       0.80      0.86      0.83       264
      B-Im_Lieferumfang_Enthalten       0.91      0.93      0.92      1279
      B-Kompatible

EPOCH 31/ 100:  59%|█████▊    | 37/63 [00:16<00:11,  2.28it/s, loss=0.536]